In [1]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_recall_curve, average_precision_score
import cv2

In [2]:

# Load CIFAR-10 dataset
from keras.datasets import cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

2023-04-10 01:25:46.905554: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /scratch/apps/python/3.7/lib/python3.7/site-packages/cv2/../../lib64:/scratch/apps/python/3.7/lib:/opt/ohpc/pub/mpi/libfabric/1.11.2/lib:/opt/ohpc/pub/mpi/ucx-ohpc/1.9.0/lib:/opt/ohpc/pub/mpi/openmpi4-gnu9/4.0.5/lib:/opt/ohpc/pub/compiler/gcc/9.3.0/lib64
2023-04-10 01:25:46.905623: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:

# Extract SIFT features from training images
sift = cv2.SIFT_create()
train_features = []
for img in train_images:
    kp, des = sift.detectAndCompute(img, None)
    if des is not None:
        train_features.append(des)
train_features = np.vstack(train_features)

In [5]:

# Cluster features using k-means
kmeans = KMeans(n_clusters=100).fit(train_features)

In [ ]:

# Calculate visual word histograms for training images
train_histograms = []
for img in train_images:
    kp, des = sift.detectAndCompute(img, None)
    histogram = np.zeros(100)
    if des is not None:
        words = kmeans.predict(des)
        for word in words:
            histogram[word] += 1
    train_histograms.append(histogram)
train_histograms = np.vstack(train_histograms)

In [ ]:

# Normalize histograms
train_histograms = normalize(train_histograms, norm='l2')

# Train classifier
clf = LinearSVC().fit(train_histograms, train_labels)

In [ ]:
# Extract SIFT features from test image
test_feature = []
kp, des = sift.detectAndCompute(test_images[0], None)
test_feature.append(des)
test_feature = np.vstack(test_feature)

In [ ]:

# Calculate visual word histogram for test image
test_histogram = np.zeros(100)
if test_feature is not None:
    words = kmeans.predict(test_feature)
    for word in words:
        test_histogram[word] += 1

In [ ]:

# Normalize histogram
test_histogram = normalize(test_histogram.reshape(1, -1), norm='l2')

# Predict class of test image
predicted_class = clf.predict(test_histogram)


In [ ]:


# Retrieve top-5 most similar images to test image
test_histograms = np.vstack([test_histogram]*len(train_images))
distances = np.linalg.norm(train_histograms - test_histograms, axis=1)
sorted_indices = np.argsort(distances)[:5]
similar_images = train_images[sorted_indices]

In [ ]:


# Calculate precision, recall, and average precision
precision, recall, _ = precision_recall_curve(test_labels, predicted_class)
ap = average_precision_score(test_labels, predicted_class)

In [ ]:

# Plot precision-recall curve
import matplotlib.pyplot as plt
plt.plot(recall, precision, color='b', label='Precision-Recall curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('Precision-Recall for CIFAR-10')
plt.legend(loc='lower left')
plt.show()

print("Precision: ", precision)
print("Recall: ", recall)
print("AP: ", ap)
